In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


train_main = pd.read_csv('S:/Hackathons/TCS/train_main.csv')
train_main.drop(columns = ['proj_id','cand_id'],inplace = True)

In [2]:
train_main.head()

,is_same_loc,mode,numb_comm_skills,top_score,eulc_dist,jacard_score_b/w_skills,result
0,1,0,1,0.2,0.072502,0.166667,SELECTED
1,0,0,0,0.2,0.111115,0.000000,SELECTED
2,1,0,0,0.1,0.067958,0.000000,SELECTED
3,1,0,0,0.2,0.084265,0.000000,SELECTED
4,1,0,1,0.4,0.105762,0.200000,REJECTED


In [3]:

from sklearn.model_selection import train_test_split
X_tr,X_tst,y_tr,y_tst = train_test_split(train_main.drop(columns = ['result']),train_main['result'],
                                         stratify = train_main['result'])

In [4]:
import xgboost

params_xg = {
        'n_estimators': [750,1000,1250,1500,2000],
        'max_depth':[7,15,20],
        'min_child_weight': [3, 10],
        'gamma': [0.5,2, 5],
        'subsample': [0.6, 0.8],
        'colsample_bytree': [0.7, 1.0]
        }


from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,log_loss,accuracy_score
score_card = pd.Series()

def best_grid_model(estimator,X_tr,X_tst,y_tr,y_tst,parmas_grid):
    
    grid = GridSearchCV(estimator=estimator,
                    param_grid=parmas_grid,
                    cv=3,  
                    verbose=10, 
                    n_jobs=-1,
                    scoring='accuracy',
                    refit = True)
    
    grid.fit(X_tr,y_tr)
    print('Best Parmaters are :',grid.best_params_)
    print('ReFitting Best Parameters to New Estimator ...............','\n')
    print('Best Score is :',grid.best_score_)
    best_estimator = estimator
    best_estimator.set_params(**grid.best_params_)
    best_estimator.fit(X_tr,y_tr)
    
    
    best_accuracy_score = accuracy_score(y_tst,best_estimator.predict(X_tst))
    print('RMSE for X_test is :',best_accuracy_score,'\n' )
    
    score_card['best_{}'.format(estimator)] = best_accuracy_score
    
    return grid,best_estimator

<ipython-input-4-7727a48eee29>:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  score_card = pd.Series()


In [5]:
grid_xgb,best_xgb = best_grid_model(xgboost.XGBClassifier(),X_tr,X_tst,y_tr,y_tst,params_xg)

Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   36.8s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   48.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   56.7s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1

[03:30:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best Parmaters are : {'colsample_bytree': 0.7, 'gamma': 5, 'max_depth': 7, 'min_child_weight': 10, 'n_estimators': 1000, 'subsample': 0.8}
ReFitting Best Parameters to New Estimator ............... 

Best Score is : 0.5765379113018598
[03:30:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RMSE for X_test is : 0.5579399141630901 



In [6]:
test_main = pd.read_csv('S:/Hackathons/TCS/test_main.csv')
best_xgb.predict_proba(test_main)


array([[0.56963426, 0.43036574],
       [0.42179763, 0.57820237],
       [0.5355673 , 0.46443272],
       ...,
       [0.44053555, 0.55946445],
       [0.5949291 , 0.40507087],
       [0.5182581 , 0.48174194]], dtype=float32)

In [7]:
best_xgb.classes_

array(['REJECTED', 'SELECTED'], dtype=object)

In [8]:
preds = pd.DataFrame(best_xgb.predict_proba(test_main),columns = best_xgb.classes_)
preds.to_csv('S:/Hackathons/TCS/preds1.csv',index = False)